In [48]:
import findspark

In [49]:
findspark.init('/home/abraham/spark-2.2.1-bin-hadoop2.7')

In [50]:
import pyspark
import os
import sys
from pyspark.sql import SQLContext
from pyspark import SparkContext
sc =SparkContext()
sqlContext = SQLContext(sc) 

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=pyspark-shell, master=local[*]) created by __init__ at <ipython-input-3-478c02255bf2>:6 

In [51]:
#pyspark 
from pyspark.ml.linalg import Vectors
from pyspark.ml.classification import LogisticRegression

# En Spark se puede crear un DataFrame de un RDD, de una lista o de un DataFrame de Pandas, 
# aquí lo estamos creando con una lista que contiene tuplas de (label, features)
# tal cual lo hacíamos en sklearn, y le estamos agregando los nombres de cada columna.
# Vectors.dense recibe una lista como parámetro
# Este DataFrame lo estamos ocupando como nuestro set de entrenamiento mock!
training = sqlContext.createDataFrame([
    (1.0, Vectors.dense([0.0, 1.1, 0.1])),
    (0.0, Vectors.dense([2.0, 1.0, -1.0])),
    (0.0, Vectors.dense([2.0, 1.3, 1.0])),
    (1.0, Vectors.dense([0.0, 1.2, -0.5]))], ["label", "features"])

In [52]:
training.show()

+-----+--------------+
|label|      features|
+-----+--------------+
|  1.0| [0.0,1.1,0.1]|
|  0.0|[2.0,1.0,-1.0]|
|  0.0| [2.0,1.3,1.0]|
|  1.0|[0.0,1.2,-0.5]|
+-----+--------------+



In [5]:
# Como lo hacíamos en sklearn, primero instanciamos el modelo que quremos
# ocupar con los parámetros que nosotros queremos tener para este 
# modelo en particular --configuramos el modelo--
lr = LogisticRegression(maxIter=10, regParam=0.01)
# Veamos la documentacion del modelo y que parametros le pusimos a nuestra
# configuracion
print("LogisticRegression parameters:\n" + lr.explainParams() + "\n")

LogisticRegression parameters:
aggregationDepth: suggested depth for treeAggregate (>= 2). (default: 2)
elasticNetParam: the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty. (default: 0.0)
family: The name of family which is a description of the label distribution to be used in the model. Supported options: auto, binomial, multinomial (default: auto)
featuresCol: features column name. (default: features)
fitIntercept: whether to fit an intercept term. (default: True)
labelCol: label column name. (default: label)
maxIter: max number of iterations (>= 0). (default: 100, current: 10)
predictionCol: prediction column name. (default: prediction)
probabilityCol: Column name for predicted class conditional probabilities. Note: Not all models output well-calibrated probability estimates! These probabilities should be treated as confidences, not precise probabilities. (default: probability)
rawPredictionCol: raw predi

In [6]:

# Ocupemos el modelo que configuramos para entrenar con lo datos que
# creamos en el DataFrame training
model_1 = lr.fit(training)

In [7]:

# model_1 es un transfomer creado a traves de un estimador (LogisticRegression)
print("Model 1 was fit using parameters: ")
# aqui estamos obteniendo la configuracion con la que se entreno
# la regresion logistica que ocupamos
print(lr.extractParamMap())


Model 1 was fit using parameters: 
{Param(parent='LogisticRegression_40dca01b6c300d3eb3cc', name='maxIter', doc='max number of iterations (>= 0).'): 10, Param(parent='LogisticRegression_40dca01b6c300d3eb3cc', name='rawPredictionCol', doc='raw prediction (a.k.a. confidence) column name.'): 'rawPrediction', Param(parent='LogisticRegression_40dca01b6c300d3eb3cc', name='fitIntercept', doc='whether to fit an intercept term.'): True, Param(parent='LogisticRegression_40dca01b6c300d3eb3cc', name='threshold', doc='Threshold in binary classification prediction, in range [0, 1]. If threshold and thresholds are both set, they must match.e.g. if threshold is p, then thresholds must be equal to [1-p, p].'): 0.5, Param(parent='LogisticRegression_40dca01b6c300d3eb3cc', name='tol', doc='the convergence tolerance for iterative algorithms (>= 0).'): 1e-06, Param(parent='LogisticRegression_40dca01b6c300d3eb3cc', name='labelCol', doc='label column name.'): 'label', Param(parent='LogisticRegression_40dca01b

In [8]:
# Tambien podemos especificar los parametros con los que queremos que 
# corra el modelo utilizando el diccionario de ParamMap
# Creamos un diccionario -se puede llamar como quieras!- que tenga
# como llave el nombre del parametro que quieres modificar, con el valor
# correspondiente.
param_map = {lr.maxIter: 20}
# Si el valor ya existe en el diccionario puedes actualizarlo
param_map[lr.maxIter] = 30  
# Tambien puedes actualizar varios parametros del diccionario al mismo tiempo
param_map.update({lr.regParam: 0.1, lr.threshold: 0.55}) 

In [9]:

# Se pueden combinar diferentes diccionarios...realmente puedes tener
# un solo diccionario con los parametros de diversos modelos que ocupes en el
# pipeline sin ningun problema, pues el valor asociado es por objeto (ID)
# aqui estamos cambiando el nombre de la columna que guarda la salida del
# modelo, por default se llama 'probability' -> verificar documentacion del 
# metodo
param_map_2 = {lr.probabilityCol: "my_probability"}  
param_map_combined = param_map.copy()
param_map_combined.update(param_map_2)
#puedes ver el contenido del diccionario con param_map_combined.items() -> python 3.5.2


In [10]:
# Entrenemos una segunda regresion logistica con los nuevos parametros que 
# establecimos a traves del paramMap
# En este fit estamos enviando tanto los datos como los parametros a ocupar en el
# modelo de regresion logistica
model_2 = lr.fit(training, param_map_combined)
print("Model 2 was fit using parameters: ")
# aqui queremos ver cono que parametros se quedo configurado el modelo
# que ocupamos para entrenar
print(lr.extractParamMap())

Model 2 was fit using parameters: 
{Param(parent='LogisticRegression_40dca01b6c300d3eb3cc', name='maxIter', doc='max number of iterations (>= 0).'): 10, Param(parent='LogisticRegression_40dca01b6c300d3eb3cc', name='rawPredictionCol', doc='raw prediction (a.k.a. confidence) column name.'): 'rawPrediction', Param(parent='LogisticRegression_40dca01b6c300d3eb3cc', name='fitIntercept', doc='whether to fit an intercept term.'): True, Param(parent='LogisticRegression_40dca01b6c300d3eb3cc', name='threshold', doc='Threshold in binary classification prediction, in range [0, 1]. If threshold and thresholds are both set, they must match.e.g. if threshold is p, then thresholds must be equal to [1-p, p].'): 0.5, Param(parent='LogisticRegression_40dca01b6c300d3eb3cc', name='tol', doc='the convergence tolerance for iterative algorithms (>= 0).'): 1e-06, Param(parent='LogisticRegression_40dca01b6c300d3eb3cc', name='labelCol', doc='label column name.'): 'label', Param(parent='LogisticRegression_40dca01b

In [11]:
# Creemos el data frame que tendra los datos de prueba mock!
test = sqlContext.createDataFrame([
    (1.0, Vectors.dense([-1.0, 1.5, 1.3])),
    (0.0, Vectors.dense([3.0, 2.0, -0.1])),
    (1.0, Vectors.dense([0.0, 2.2, -1.5]))], ["label", "features"])

In [12]:
# Make predictions on test data using the Transformer.transform() method.
# LogisticRegression.transform will only use the 'features' column.
# Note that model2.transform() outputs a "myProbability" column instead of the usual
# 'probability' column since we renamed the lr.probabilityCol parameter previously.
# Ahora si, hagamos predicciones sobre el set de pruebas ocupando el cerebro
# antes entrenado utilizando el transform
prediction = model_2.transform(test)
# la respuesta es un DataFrame (la salida de un transform en su DataFrame)
# por lo que podemos aplicarle los metodos de SparkSQL :)
# verificamos que si es un DataFrame...
type(prediction)

pyspark.sql.dataframe.DataFrame

In [13]:
# veamos que columnas tiene este DataFrame (como el names de R)
prediction.columns
# Aqui estamos seleccionando las columnas features, label, 
# my_probability -> que es el nombre que nosotros especificamos anteriormente en
# ParamMap, y la columna prediction que es el nombre por default que regresa
# el modelo al parametro 'predictionCol' -> ver documentacion
# el collect hara que se regresen los resultados al drive!!! 
result = prediction.select("features", "label", "my_probability", "prediction") \
    .collect()


In [14]:
for row in result:
    print("features={}, label={} -> prob={}, prediction={}".format( \
    row.features, row.label, row.my_probability, row.prediction))

features=[-1.0,1.5,1.3], label=1.0 -> prob=[0.0570730417103,0.94292695829], prediction=1.0
features=[3.0,2.0,-0.1], label=0.0 -> prob=[0.92385223117,0.0761477688296], prediction=0.0
features=[0.0,2.2,-1.5], label=1.0 -> prob=[0.109727761148,0.890272238852], prediction=1.0


# Ejemplo 2

In [15]:
#pyspark
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import HashingTF, Tokenizer


In [16]:
# al igual que en el ejemplo anterior, creamos un dataframe a traves
# de una lista con los datos de entrenamiento, la lista esta formada
# por tuplas (id, texto, label). Esta forma no nos servirá para poder meterla
# en los objetos de ML, pero mas adelante arreglaremos esto
training = sqlContext.createDataFrame([
    (0, "a b c d e spark", 1.0),
    (1, "b d", 0.0),
    (2, "spark f g h", 1.0),
    (3, "hadoop mapreduce", 0.0)
], ["id", "text", "label"])

In [17]:

# Definimos los transformers: Tokenizer y HashingTF, y los 
# etimators: LogisticRegression que ocuparemos. Nota que aqui no hemos hecho
# ningun fit todavia... la magia vendra mas adelante ;)
# Tokenizer convierte el string de entrada (inputCol) a minusculas y separa en
# palabras utilizando ocmo separador el espacio
tokenizer = Tokenizer(inputCol="text", outputCol="words")
# HashingTF permite hashear cada palabra utilizando MurmurHash3 convirtiendo
# el hash generado en el indice a poner en el "TDM". Este metodo optimiza el
# tiempo para generar el TDM de TF-IDF "normal". Para evitar colisiones en
# la conversion a hash se aumenta el numero de bucket -se recomienda ocupar
# potencias de 2 para balancear las cubetas-
# Nota que en este transformet estamos ocupando como entrada la salida del
# transformer Tokenizer
hashingTF = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol="features")
# Ocuparemos una regresion logistica de nuex
lr = LogisticRegression(maxIter=10, regParam=0.001)
# Aqui viene lo bonito... definimos un pipeline que tiene como etapas/pasos
# primero el tokenizer, luego el hasing y luego la regresion logistica. Aqui
# estamos definiendo el flujo de procesamiento, el DAG! 
pipeline = Pipeline(stages=[tokenizer, hashingTF, lr])

In [18]:
# Voila, solo se requiere de hacer fit al pipeline para que esto funcione
# como un pipeline, siguiendo el orden de los pasos establecidos en la 
# definicion del pipeline :) ... recuerda que el fit solo es como 
# el entrenamiento una vez que ya definimos las configuraciones de 
# los objetos que ocuparemos (transformers y estimators)
model = pipeline.fit(training)

In [19]:
# Creamos el dataframe de pruebas mock! -> Nota que aqui no hay 
# label!!!! (asi funcionaria en produccion cierto!)
test = sqlContext.createDataFrame([
    (4, "spark i j k"),
    (5, "l m n"),
    (6, "spark hadoop spark"),
    (7, "apache hadoop")
], ["id", "text"])


In [20]:
# Lixto, "ejecutamos" el pipeline haciendo un transform al pipeline para 
# obtener las predicciones del set de pruebas
prediction = model.transform(test)

In [21]:
# De nuevo, prediction es un DataFrame generado con un transformer generado
# a traves de estimadores y transformers :) 
# Seleccionamos las columnas que queremos ver 
selected = prediction.select("id", "text", "probability", "prediction")
for row in selected.collect():
    rid, text, prob, prediction = row
    print("({}, {}) --> prob={}, prediction={}".format( \
    rid, text, str(prob), prediction))


(4, spark i j k) --> prob=[0.159640773879,0.840359226121], prediction=1.0
(5, l m n) --> prob=[0.837832568548,0.162167431452], prediction=0.0
(6, spark hadoop spark) --> prob=[0.0692663313298,0.93073366867], prediction=1.0
(7, apache hadoop) --> prob=[0.982157533344,0.0178424666556], prediction=0.0


In [22]:
#pyspark
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, CountVectorizer


In [23]:
# Creamos nuestro set de entrada para formar la TDM
sentence_data = sqlContext.createDataFrame([
    (0.0, "Hi I heard about Spark"),
    (0.0, "I wish Java could use case classes"),
    (1.0, "Logistic regression models are neat")
], ["label", "sentence"])

In [24]:
# Ocupamos el transformer Tokenizer para separar por palabras
tokenizer = Tokenizer(inputCol="sentence", outputCol="words")
# Aqui no hay train! porque no estamos entrenando nanda... estamos en un problema
# de IR. Tokenizer no tiene un metodo fit -no hay entrenamiento-
words_data = tokenizer.transform(sentence_data)

In [25]:
# Ocupamos el transformer CountVectorizer para generar una matriz de 
# terminos y sus frecuencias 
count_vectorizer = CountVectorizer(inputCol="words", outputCol="raw_features")
featurized_model = count_vectorizer.fit(words_data)
featurized_data = featurized_model.transform(words_data)
featurized_data.show(truncate=False)

+-----+-----------------------------------+------------------------------------------+----------------------------------------------------+
|label|sentence                           |words                                     |raw_features                                        |
+-----+-----------------------------------+------------------------------------------+----------------------------------------------------+
|0.0  |Hi I heard about Spark             |[hi, i, heard, about, spark]              |(16,[0,4,8,9,13],[1.0,1.0,1.0,1.0,1.0])             |
|0.0  |I wish Java could use case classes |[i, wish, java, could, use, case, classes]|(16,[0,2,5,6,7,11,15],[1.0,1.0,1.0,1.0,1.0,1.0,1.0])|
|1.0  |Logistic regression models are neat|[logistic, regression, models, are, neat] |(16,[1,3,10,12,14],[1.0,1.0,1.0,1.0,1.0])           |
+-----+-----------------------------------+------------------------------------------+----------------------------------------------------+



In [26]:
# Ocupamos IDF para obtener el IDF de la coleccion de documentos mock que 
# generamos. IDF si tiene un metodo fit a traves del cual le enviamos el set 
# de tokens al que queremos obtener el IDF
idf = IDF(inputCol="raw_features", outputCol="features", minDocFreq=1)
# Aqui obtenemos el modelo a ocupar (transformer) a ocupar 
idf_model = idf.fit(featurized_data)
rescaled_data = idf_model.transform(featurized_data)

rescaled_data.select("label", "features").show(truncate=False)

+-----+--------------------------------------------------------------------------------------------------------------------------------------------------------------+
|label|features                                                                                                                                                      |
+-----+--------------------------------------------------------------------------------------------------------------------------------------------------------------+
|0.0  |(16,[0,4,8,9,13],[0.28768207245178085,0.6931471805599453,0.6931471805599453,0.6931471805599453,0.6931471805599453])                                           |
|0.0  |(16,[0,2,5,6,7,11,15],[0.28768207245178085,0.6931471805599453,0.6931471805599453,0.6931471805599453,0.6931471805599453,0.6931471805599453,0.6931471805599453])|
|1.0  |(16,[1,3,10,12,14],[0.6931471805599453,0.6931471805599453,0.6931471805599453,0.6931471805599453,0.6931471805599453])                                          

In [27]:
#pyspark
from pyspark.ml.feature import OneHotEncoder, StringIndexer

# creamos nuestro set de datos de entrada categorico
df = sqlContext.createDataFrame([
    (0, "a"),
    (1, "b"),
    (2, "c"),
    (3, "a"),
    (4, "a"),
    (5, "c")
], ["id", "category"])

# Esta funcion agrega un id numerico a cada valor diferente de un valor categorico 
# es como establecer los niveles en R de una factor pero los niveles son numericos,
# sus id. El indice se establece por orden de frecuencia (descendente), por lo que 
# el indice 0 corresponde a la variable que aparece con mas frecuencia
string_indexer = StringIndexer(inputCol="category", outputCol="categoryIndex")
model = string_indexer.fit(df)
indexed = model.transform(df)
indexed.show()

# OneHotEncoder no tiene un fit ya que solo es un transformador
encoder = OneHotEncoder(inputCol="categoryIndex", outputCol="categoryVec")
encoded = encoder.transform(indexed)
encoded.show()

+---+--------+-------------+
| id|category|categoryIndex|
+---+--------+-------------+
|  0|       a|          0.0|
|  1|       b|          2.0|
|  2|       c|          1.0|
|  3|       a|          0.0|
|  4|       a|          0.0|
|  5|       c|          1.0|
+---+--------+-------------+

+---+--------+-------------+-------------+
| id|category|categoryIndex|  categoryVec|
+---+--------+-------------+-------------+
|  0|       a|          0.0|(2,[0],[1.0])|
|  1|       b|          2.0|    (2,[],[])|
|  2|       c|          1.0|(2,[1],[1.0])|
|  3|       a|          0.0|(2,[0],[1.0])|
|  4|       a|          0.0|(2,[0],[1.0])|
|  5|       c|          1.0|(2,[1],[1.0])|
+---+--------+-------------+-------------+



In [28]:
#pyspark
from pyspark.ml.feature import MinMaxScaler
from pyspark.ml.linalg import Vectors

data_frame = sqlContext.createDataFrame([
    (0, Vectors.dense([1.0, 0.1, -1.0]),),
    (1, Vectors.dense([2.0, 1.1, 1.0]),),
    (2, Vectors.dense([3.0, 10.1, 3.0]),)
], ["id", "features"])
data_frame.show()

# Configuramos el estimator MinMaxScaler como lo necesitamos
scaler = MinMaxScaler(inputCol="features", outputCol="scaled_features")

# Creamos el modelo MinMaxScaler (transformer)
scaler_model = scaler.fit(data_frame)

# Transformamos los datos reescalando 
scaled_data = scaler_model.transform(data_frame)
# Nota que cuando pedimos getMin y getMax lo hacemos al estimator, no al modelo
print("Features scaled to range: [{}, {}]".format(scaler.getMin(), scaler.getMax()))
scaled_data.select("features", "scaled_features").show(truncate=False)


+---+--------------+
| id|      features|
+---+--------------+
|  0|[1.0,0.1,-1.0]|
|  1| [2.0,1.1,1.0]|
|  2|[3.0,10.1,3.0]|
+---+--------------+

Features scaled to range: [0.0, 1.0]
+--------------+---------------+
|features      |scaled_features|
+--------------+---------------+
|[1.0,0.1,-1.0]|[0.0,0.0,0.0]  |
|[2.0,1.1,1.0] |[0.5,0.1,0.5]  |
|[3.0,10.1,3.0]|[1.0,1.0,1.0]  |
+--------------+---------------+



In [29]:
#pyspark
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import StandardScaler

# Creamos el data frame que queremos estandarizar
data_frame = sqlContext.createDataFrame([
    (0, Vectors.dense([1.0, 0.1, -1.0]),),
    (1, Vectors.dense([2.0, 1.1, 1.0]),),
    (2, Vectors.dense([3.0, 10.1, 3.0]),)
], ["id", "features"])
# Configuramos el estimator StandarScaler como lo necesitamos (por default
# withMean esta en False porque hace que se regrese un vector dense...
# hay que tener cuidado con eso cuando estemos manejandoo vectores sparse
scaler = StandardScaler(inputCol="features", outputCol="scaled_features",
                        withStd=True, withMean=True)
# Creamos el modelo StandardScaler para los datos de entrada
scaler_model = scaler.fit(data_frame)

# Transformamos los datos 
scaled_data = scaler_model.transform(data_frame)
scaled_data.show(truncate=False)

+---+--------------+-------------------------------+
|id |features      |scaled_features                |
+---+--------------+-------------------------------+
|0  |[1.0,0.1,-1.0]|[-1.0,-0.6657502859356826,-1.0]|
|1  |[2.0,1.1,1.0] |[0.0,-0.4841820261350419,0.0]  |
|2  |[3.0,10.1,3.0]|[1.0,1.1499323120707245,1.0]   |
+---+--------------+-------------------------------+



In [30]:
from pyspark.ml.feature import ChiSqSelector
from pyspark.ml.linalg import Vectors

# creamos nuestro set de datos con features
df = sqlContext.createDataFrame([
    (7, Vectors.dense([0.0, 0.0, 18.0, 1.0]), 1.0,),
    (8, Vectors.dense([0.0, 1.0, 12.0, 0.0]), 0.0,),
    (9, Vectors.dense([1.0, 0.0, 15.0, 0.1]), 0.0,)], ["id", "features", "clicked"])

# Configuramos el estimator
selector = ChiSqSelector(numTopFeatures=1, featuresCol="features",
                         outputCol="selectedFeatures", labelCol="clicked")
# Creamos el modelo de ChiSquare y luego ocupamos el transform para 
# seleccionar los features con mas informacion
result = selector.fit(df).transform(df)

print("ChiSqSelector output with top {} features selected".format( \
selector.getNumTopFeatures()))
result.show()

ChiSqSelector output with top 1 features selected
+---+------------------+-------+----------------+
| id|          features|clicked|selectedFeatures|
+---+------------------+-------+----------------+
|  7|[0.0,0.0,18.0,1.0]|    1.0|          [18.0]|
|  8|[0.0,1.0,12.0,0.0]|    0.0|          [12.0]|
|  9|[1.0,0.0,15.0,0.1]|    0.0|          [15.0]|
+---+------------------+-------+----------------+



# pca

In [32]:
from pyspark.ml.feature import PCA
from pyspark.ml.linalg import Vectors

# Creamos el data frame de 5 dimensiones, nota que en este data frame estamos
# ocupando un vector de tipo sparse en donde solo ponemos la dimension del vector
# los indices donde los valores son diferentes de 0 y los valores de esos indices
data = [(Vectors.sparse(5, [(1, 1.0), (3, 7.0)]),),#vector de 5 elementos
        (Vectors.dense([2.0, 0.0, 3.0, 4.0, 5.0]),),
        (Vectors.dense([4.0, 0.0, 0.0, 6.0, 7.0]),)]
df = sqlContext.createDataFrame(data, ["features"])

# Configuramos el modelo 
pca = PCA(k=3, inputCol="features", outputCol="pcaFeatures")
# Creamos el estimator PCA
model = pca.fit(df)
# Transformamos a PCA los valores quedandonos con los primeros 3 componentes
result = model.transform(df).select("pcaFeatures")
result.show(truncate=False)

+-----------------------------------------------------------+
|pcaFeatures                                                |
+-----------------------------------------------------------+
|[1.6485728230883807,-4.013282700516296,-5.524543751369388] |
|[-4.645104331781534,-1.1167972663619026,-5.524543751369387]|
|[-6.428880535676489,-5.337951427775355,-5.524543751369389] |
+-----------------------------------------------------------+



In [34]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Cargamos los datos de prueba
data = sqlContext.read.format("libsvm").load("sample_libsvm_data.txt")


AnalysisException: 'Path does not exist: file:/home/abraham/MGE_2018/sample_libsvm_data.txt;'

In [36]:
#pyspark
from pyspark.ml.clustering import KMeans

# Creamos el data frame con los datos que ocuparemos mock
dataset = sqlContext.createDataFrame([
    (0, Vectors.dense([0.0, 0.0, 0])),
    (1, Vectors.dense([0.1, 0.1, 0.1])),
    (2, Vectors.dense([0.2, 0.2, 0.2])),
    (3, Vectors.dense([9, 9, 9])),
    (4, Vectors.dense([9.1, 9.1, 9.1])),
    (5, Vectors.dense([9.2, 9.2, 9.2]))], ["label", "features"])

# Configuramos  el KMeans con 2 grupos
kmeans = KMeans().setK(2).setSeed(1)
# Creamos el estimator Kmeans 
model = kmeans.fit(dataset)

# Vemos que tan bien o mal nos fue, obteniendo el SSE de los puntos a
# su centro
wssse = model.computeCost(dataset)
print("Within Set Sum of Squared Errors = " + str(wssse))

# Obtenemos centros 
centers = model.clusterCenters()
print("Cluster Centers: ")
for center in centers:
    print(center)

Within Set Sum of Squared Errors = 0.11999999999994547
Cluster Centers: 
[ 9.1  9.1  9.1]
[ 0.1  0.1  0.1]


# Hypr

In [37]:
#pyspark
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.feature import HashingTF, Tokenizer
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

# Creamos un DataFrame con ids, texto, label
training = sqlContext.createDataFrame([
    (0, "a b c d e spark", 1.0),
    (1, "b d", 0.0),
    (2, "spark f g h", 1.0),
    (3, "hadoop mapreduce", 0.0),
    (4, "b spark who", 1.0),
    (5, "g d a y", 0.0),
    (6, "spark fly", 1.0),
    (7, "was mapreduce", 0.0),
    (8, "e spark program", 1.0),
    (9, "a e c l", 0.0),
    (10, "spark compile", 1.0),
    (11, "hadoop software", 0.0)
], ["id", "text", "label"])

# Configuramos los elementos que formaran parte de nuestro pipeline: Tokenizer, HashingTF,
# regresion logística y pipeline
tokenizer = Tokenizer(inputCol="text", outputCol="words")
hashingTF = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol="features")
lr = LogisticRegression(maxIter=10)
pipeline = Pipeline(stages=[tokenizer, hashingTF, lr])

In [38]:
# Definimos uno de los 3 elementos que necesitamos para hacer hpyperparameter tuning
# el grid de parametros.
# Especificamos el grid que queremos explorar agregando los parametros
# que queremos modificar de cada estimator y poniendo los valores que 
# queremos explorar para cada uno de ellos en una lista. 
# En este grid estamos estableciendo que queremos explorar tener 10
# cubetas, 100 y 1000 (siempre ocupar valores pares para que las cubetas
# queden balanceadas); para la regresion logistica ocupamos dos parametros de 
# regularizacion
paramGrid = ParamGridBuilder() \
    .addGrid(hashingTF.numFeatures, [10, 100, 1000]) \
    .addGrid(lr.regParam, [0.1, 0.01]) \
    .build()

In [39]:
# Definimos el estimador que queremos evaluar -recuerda que pipeline es un estimator-
# y el evaluador que ocuparemos para determinar el desempenio de cada modelo
# por default el BinaryClassificationEvaluator toma como metrica de desempenio 
# el AUC -metricName-, solo hay AUC o area under precidion recall curve
crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=BinaryClassificationEvaluator(),#default es AUC
                          numFolds=2)  # ocupar minimo 3 - 10 normalmente!

In [45]:
# Cross validation tambien es un estimador, le hacemos fit
cvModel = crossval.fit(training)
# veamos cual fue el mejor modelo
cvModel.bestModel

PipelineModel_4fb597d45f07c2023703

In [43]:
# Creamos nuestro dataframe de pruebas
test = sqlContext.createDataFrame([
    (4, "spark i j k"),
    (5, "l m n"),
    (6, "mapreduce spark"),
    (7, "apache hadoop")
], ["id", "text"])

# Hacemos predicciones con el mejor modelos encontrado por el CrossValidator que es el
# que quedo en cvModel
prediction = cvModel.transform(test)
# seleccionamos las columnas que queremos ver y las imprimimos para ver el resultado
selected = prediction.select("id", "text", "probability", "prediction")
for row in selected.collect():
    print(row)

Row(id=4, text='spark i j k', probability=DenseVector([0.2581, 0.7419]), prediction=1.0)
Row(id=5, text='l m n', probability=DenseVector([0.9186, 0.0814]), prediction=0.0)
Row(id=6, text='mapreduce spark', probability=DenseVector([0.432, 0.568]), prediction=1.0)
Row(id=7, text='apache hadoop', probability=DenseVector([0.6766, 0.3234]), prediction=0.0)


In [47]:
#pyspark
from pyspark.streaming import StreamingContext

#sc = spark.sparkContext
sc

# requerimos el sparkContext de nuestro sparkSession e indicarle
# el intervalo de tiempo para hacer cortes en el streaming y generar
# los batches de datos
ss = StreamingContext(sc, 1)
ss